In [ ]:
import random
import json
import requests
import csv
import pandas as pd
import math
import numpy as np
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

a = pd.read_csv("20knot_wide.csv", encoding="shift_jis")#widly_20_Sym
strip_data =a.values
b = pd.read_csv("wave_statistics.csv", encoding="shift_jis")#widly_20_Sym
wave_statistics =b.values

n_fruid  = 1    #速度６種類
n_angle  = 72   #0~350まで36パターン 0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,
n_lambda = 80   #波長40パターン
Iy = 2661319053 #mm^2*m^2
y_y = 17.612 #[m] 中立軸
Zc=66.122*10**6
kissui=15.7
    
def main():
    n_gene   = 62   # ビット数
    n_ind    = 1    # 群サイズ   
    print("start")
 
    f=open("itiyoubunnpu_ver1.csv","w",encoding="UTF-8")
    #f.write("入力32で出力5の1峰海象,高周波除く、船速20ノットとsは周波数依存、ωは波高により取りうる値の範囲が変化,波向きはcosとsinで出力。出力5だが、sが無くなり、波向きがcosとsinの二つに増えた事に注意、AG列からが出力データ")  
    #f.write('\n')
    
    omegalist=[]
    for i in range(n_lambda):
        omegalist.append(strip_data[i][3])
        
    encount_omega_2 = list(np.arange(0, 2.11,0.03 )) 
    #encount_omega_2.append(2*encount_omega_2[len(encount_omega_2)-1]-encount_omega_2[len(encount_omega_2)-2])    
    for l in range(40000):
        random.seed()
        lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2 = get_parameter_from_ind() #出力データのスペクトルパラメータ
        
        S_max1=def_S_max(Hv_1,omega_1)
        S_max2=0#def_S_max(Hv_2,omega_2)       
        for k in range(1): #6フルード数ごと 0,5,10,15,20,25
            Pitch_S_e_data = []
            Roll_S_e_data  = []
            Heave_S_e_data = []
            load_S_e_data=[]
            Pitch_Roll_S_e_data    = []
            #Roll_Pitch_S_e_data    = []
            Roll_Heave_S_e_data    = []
            #Heave_Roll_S_e_data    = []
            Heave_Pitch_S_e_data   = []
            #Pitch_Heave_S_e_data   = []
            load_Roll_S_e_data    = []
            #Roll_load_S_e_data    = []
            load_Heave_S_e_data    = []
            #Heave_load_S_e_data    = []
            load_Pitch_S_e_data   = []
            #Pitch_load_S_e_data   = []
            Fn    = strip_data[n_lambda*n_angle*k][0]#フルード数
            
            #S_data = list(np.linspace(0,0,len(omegalist)))
            
            for j in range(n_angle): #36の角度 0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,
                encount_angle = strip_data[n_lambda*j+n_lambda*n_angle*k][1]#波向き
                Pitch_S_data_by_lambda =[]
                Roll_S_data_by_lambda  =[]
                Heave_S_data_by_lambda =[]
                load_S_data_by_lambda =[]
                Pitch_Roll_S_data_by_lambda  =[]
                #Roll_Pitch_S_data_by_lambda  =[]
                Roll_Heave_S_data_by_lambda  =[]
                #Heave_Roll_S_data_by_lambda  =[]
                Heave_Pitch_S_data_by_lambda =[]
                #Pitch_Heave_S_data_by_lambda =[]
                load_Roll_S_data_by_lambda   =[]
                #Roll_load_S_data_by_lambda   =[]
                load_Heave_S_data_by_lambda  =[]
                #Heave_load_S_data_by_lambda  =[]
                load_Pitch_S_data_by_lambda  =[]
                #Pitch_load_S_data_by_lambda  =[]
                
                for i in range(n_lambda): #40周波数ごと
                    line          = i+n_lambda*j+n_lambda*n_angle*k    #行
                    wave_spectrum =spectrum_by_angle(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,omegalist[i],encount_angle,S_max1,S_max2)
                    
                    """  Pitch  """ #ここで全角度分のスペクトル関数をつくる。ω_iでの値をどんどん足し合わせる
                    Pitch_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 13, 13)),0])   #前者が共役の複素数。13はPitch振幅の列
                    Roll_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 11, 11)),0])
                    Heave_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 9, 9)),0])
                    load_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 17, 17)*(1000*9.8065/Zc)**2),0])
                    #Pitch_Roll_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 13, 11)),0])
                    Pitch_Roll_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 13, 11).real,KeisanResponse(wave_spectrum , line , 13, 11).imag])
                    #Roll_Pitch_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 11, 13).real,KeisanResponse(wave_spectrum , line , 11, 13).imag])
                    #Roll_Heave_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 11, 9)),0])
                    Roll_Heave_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 11, 9).real,KeisanResponse(wave_spectrum , line , 11, 9).imag])
                    #Heave_Roll_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 9, 11).real,KeisanResponse(wave_spectrum , line , 9, 11).imag])
                    #Heave_Pitch_S_data_by_lambda.append([np.abs(KeisanResponse(wave_spectrum , line , 9, 13)),0])
                    Heave_Pitch_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 9, 13).real,KeisanResponse(wave_spectrum , line , 9, 13).imag])
                    #Pitch_Heave_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 13, 9).real,KeisanResponse(wave_spectrum , line , 13, 9).imag])
                    load_Roll_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 17, 11).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 17, 11).imag*(1000*9.8065/Zc)])
                    #Roll_load_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 11, 17).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 11, 17).imag*(1000*9.8065/Zc)])
                    load_Heave_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 17, 9).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 17, 9).imag*(1000*9.8065/Zc)])
                    #Heave_load_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 9, 17).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 9, 17).imag*(1000*9.8065/Zc)])
                    load_Pitch_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 17, 13).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 17, 13).imag*(1000*9.8065/Zc)])
                    #Pitch_load_S_data_by_lambda.append([KeisanResponse(wave_spectrum , line , 13, 17).real*(1000*9.8065/Zc),KeisanResponse(wave_spectrum , line , 13, 17).imag*(1000*9.8065/Zc)])
                                  
                    #S_data[i] += KeisanResponse(wave_spectrum , line , 13, 11)
                    if i != 0: #出会い波周波数の該当リスト
                        list_omega_j,Eomega1, Eomega2 = search_omega_i(i,omegalist,encount_omega_2,encount_angle,Fn)
                        #出会い波周波数へ変換して
                        Pitch_S_e_data.append(To_trans_Se(i,Pitch_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        Roll_S_e_data.append(To_trans_Se(i,Roll_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        Heave_S_e_data.append(To_trans_Se(i,Heave_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        load_S_e_data.append(To_trans_Se(i,load_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        Pitch_Roll_S_e_data.append(To_trans_Se(i,Pitch_Roll_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Roll_Pitch_S_e_data.append(To_trans_Se(i,Roll_Pitch_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        Roll_Heave_S_e_data.append(To_trans_Se(i,Roll_Heave_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Heave_Roll_S_e_data.append(To_trans_Se(i,Heave_Roll_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        Heave_Pitch_S_e_data.append(To_trans_Se(i,Heave_Pitch_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Pitch_Heave_S_e_data.append(To_trans_Se(i,Pitch_Heave_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        load_Roll_S_e_data.append(To_trans_Se(i,load_Roll_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Roll_load_S_e_data.append(To_trans_Se(i,Roll_load_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        load_Heave_S_e_data.append(To_trans_Se(i,load_Heave_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Heave_load_S_e_data.append(To_trans_Se(i,Heave_load_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        load_Pitch_S_e_data.append(To_trans_Se(i,load_Pitch_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
                        #Pitch_load_S_e_data.append(To_trans_Se(i,Pitch_load_S_data_by_lambda,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist))
            #f.write(str(int(Fn*math.sqrt(349.2*9.81)*3600/1852))+",")
            mp0,mp1=final_keisan(Pitch_S_e_data,encount_omega_2,f)
            mr0,mr1=final_keisan(Roll_S_e_data,encount_omega_2,f)
            mh0,mh1=final_keisan(Heave_S_e_data,encount_omega_2,f)
            ml0,ml1=final_keisan(load_S_e_data,encount_omega_2,f)
            mpr0,mpr1=X_final_keisan1(Pitch_Roll_S_e_data,encount_omega_2,f)
            #mrp0,mrp1=X_final_keisan1(Roll_Pitch_S_e_data,encount_omega_2,f)
            mrh0,mrh1=X_final_keisan1(Roll_Heave_S_e_data,encount_omega_2,f)
            #mhr0,mhr1=X_final_keisan1(Heave_Roll_S_e_data,encount_omega_2,f)
            mhp0,mhp1=X_final_keisan1(Heave_Pitch_S_e_data,encount_omega_2,f)
            #mph0,mph1=X_final_keisan1(Pitch_Heave_S_e_data,encount_omega_2,f)
            mlr0,mlr1=X_final_keisan1(load_Roll_S_e_data,encount_omega_2,f)
            #mrl0,mrl1=X_final_keisan1(Roll_load_S_e_data,encount_omega_2,f)
            mlh0,mlh1=X_final_keisan1(load_Heave_S_e_data,encount_omega_2,f)
            #mhl0,mhl1=X_final_keisan1(Heave_load_S_e_data,encount_omega_2,f)
            mlp0,mlp1=X_final_keisan1(load_Pitch_S_e_data,encount_omega_2,f)
            #mpl0,mpl1=X_final_keisan1(Pitch_load_S_e_data,encount_omega_2,f)
            #f.write(str(kissui)+",")  
            #plt.plot( omegalist, S_data , label='|F(k)|', marker='o', linestyle='-')
            #plt.xlabel('ω')
            #plt.ylabel('S(ω)')#'#θ_xy[rad]
            #plt.grid()
            #plt.show()
            #m_0 =0.0
            #for i in range(len(omegalist)-1):
                #m_0 += (S_data[i]+S_data[i+1])*(-omegalist[i+1]+omegalist[i])/2  #台形則
            #print(m_0)  
            #print("P0:"+str(mp0)+" P1:"+str(mp1)+" R0:"+str(mr0)+" R1:"+str(mr1)+" H0:"+str(mh0)+" H1:"+str(mh1)+" Lo:"+str(ml0))
            #print("PR0:"+str(mpr0)+" PR1:"+str(mpr1)+" RH0:"+str(mrh0)+" RH1:"+str(mrh1)+" HP0:"+str(mhp0)+" HP1:"+str(mhp1))
            #phase_moment(Pitch_Roll_Phase_by_lambda,omegalist,PR_S)
             
            if Hv_1>Hv_2:
                output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
            elif Hv_1<Hv_2:
                output2(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
            else:
                if theta_1>theta_2:
                    output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                elif theta_1<theta_2:
                    output2(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                else:
                    if omega_1>omega_2:
                        output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                    elif omega_1<omega_2:
                        output2(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                    else:
                        if lambda_1>lambda_2:
                            output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                        elif lambda_1<lambda_2:
                            output2(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                        else:
                            if s_1>s_2:
                                output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
                            elif s_1<=s_2:
                                output2(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f)
        print(l+1)
    f.close()
    print("end")

 
 
class MyClass(np.ndarray):
    """Container of a individual."""
    encount_omega = None
    def __new__(cls, a):
        return np.asarray(a).view(cls)


def output1(lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f):
    #print("Hv",Hv_1,"s",s_1,"theta",theta_1,"lambda",lambda_1,"w",omega_1)
    #print("Hv",Hv_2,"s",s_2,"theta",theta_2,"lambda",lambda_2,"w",omega_2)
    f.write(str('%03.6f' %(Hv_1))+","+str('%03.6f' %(math.cos(theta_1*math.pi/180)))+","+str('%03.6f' %(math.sin(theta_1*math.pi/180)))+","+str(lambda_1)+","+str('%03.6f' %(omega_1)))
    #f.write(str('%03.6f' %(Hv_2))+","+str(s_2)+","+str('%03.6f' %(theta_2))+","+str(lambda_2)+","+str('%03.6f' %(omega_2)))
    f.write('\n')
    
def output2(lambda_1,theta_1,s_1,omega_1,Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2,f):
    #print("Hv",Hv_2,"s",s_2,"theta",theta_2,"lambda",lambda_2,"w",omega_2)
    #print("Hv",Hv_1,"s",s_1,"theta",theta_1,"lambda",lambda_1,"w",omega_1)
    f.write(str('%03.6f' %(Hv_2))+","+str(s_2)+","+str('%03.6f' %(math.cos(theta_2*math.pi/180)))+","+str('%03.6f' %(math.sin(theta_2*math.pi/180)))+","+str(lambda_2)+","+str('%03.6f' %(omega_2))+",")
    f.write(str('%03.6f' %(Hv_1))+","+str(s_1)+","+str('%03.6f' %(theta_1))+","+str(lambda_1)+","+str('%03.6f' %(omega_1)))
    f.write('\n') 


def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    for i, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[i].fitness = fit
    
def search_omega_i(i,omegalist,encount_omega_2,encount_angle,Fn):
    """間に入る出会い波周波数のリスト作成"""
    Eomega1=np.abs(omegalist[i-1]*(1-Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i-1]*math.cos(encount_angle*math.pi/180)))
    Eomega2=np.abs(omegalist[i]*(1-Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i]*math.cos(encount_angle*math.pi/180)))
    Eomega_fore = max(Eomega1,Eomega2)
    Eomega_aft =  min(Eomega1,Eomega2)
    #if math.cos(encount_angle*math.pi/180) >= 9.80655/2/1.37/(Fn*math.sqrt(349.2*9.80655)): #0<ω<1.37[rad/sに頂点がある場合
    omega_vertex  = 9.80655/(2*Fn*math.sqrt(349.2*9.80655)*math.cos(encount_angle*math.pi/180)) #頂点のx座標
    if (omega_vertex - omegalist[i-1])*(omega_vertex -omegalist[i]) <= 0.000: #頂点のx座標が挟まれる場合
        Eomega_fore = omega_vertex/2 #頂点のy座標はx座標の半分だから  
    elif (omega_vertex*2 - omegalist[i-1])*(omega_vertex*2 -omegalist[i]) <= 0: #x軸との交点が挟まれる場合
        Eomega_aft = 0.0  
    
    list_omega_j=[]
    for j in range(len(encount_omega_2)):
        if (Eomega_fore-encount_omega_2[j])*(Eomega_aft-encount_omega_2[j]) <=0:
            list_omega_j.append(j)
    return list_omega_j ,Eomega1, Eomega2

def To_trans_Se(i,S,encount_angle,Fn,list_omega_j,encount_omega_2,Eomega1, Eomega2,omegalist):#Eomega1,2=Eomega_fore,aft
    """線形補間"""    
    Mi=[[0 for k in range(2)] for l in range(len(encount_omega_2))] #list(np.linspace(0, 0, len(encount_omega_2)))
    omega_vertex  = 9.80655/(2*Fn*math.sqrt(349.2*9.80655)*math.cos(encount_angle*math.pi/180)) #頂点のx座標
    for j in  range(len(list_omega_j)):
        #if math.cos(encount_angle*math.pi/180) >= 9.80655/2/1.37/(Fn*math.sqrt(349.2*9.80655)): #0<ω<1.37[rad/sに頂点がある場合
        if (omega_vertex - omegalist[i-1])*(omega_vertex -omegalist[i]) <= 0: #頂点のx座標が挟まれる場合
            Mi[list_omega_j[j]][0] += (S[i-1][0]+S[i][0])*0.003/2/(omega_vertex/2-min(Eomega1,Eomega2))
            Mi[list_omega_j[j]][1] += (S[i-1][1]+S[i][1])*0.003/2/(omega_vertex/2-min(Eomega1,Eomega2))
        elif (omega_vertex*2 - omegalist[i-1])*(omega_vertex*2 -omegalist[i]) <= 0: #x軸との交点が挟まれる場合
            Mi[list_omega_j[j]][0] += (S[i-1][0]+S[i][0])*0.003/2/max(Eomega1,Eomega2)
            Mi[list_omega_j[j]][1] += (S[i-1][1]+S[i][1])*0.003/2/max(Eomega1,Eomega2)
        else:
            dwe1 = np.abs(1.0-2.0*Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i-1]*math.cos(encount_angle*math.pi/180.0))
            dwe2 = np.abs(1.0-2.0*Fn*math.sqrt(349.2*9.80655)/9.80655*omegalist[i]*math.cos(encount_angle*math.pi/180.0))
            fore_real = (Eomega2-encount_omega_2[list_omega_j[j]])/(Eomega2-Eomega1)*(S[i-1][0])/dwe1
            aft_real  = (encount_omega_2[list_omega_j[j]]-Eomega1)/(Eomega2-Eomega1)*(S[i][0])/dwe2
            fore_imag = (Eomega2-encount_omega_2[list_omega_j[j]])/(Eomega2-Eomega1)*(S[i-1][1])/dwe1
            aft_imag  = (encount_omega_2[list_omega_j[j]]-Eomega1)/(Eomega2-Eomega1)*(S[i][1])/dwe2
            if (Eomega1-encount_omega_2[list_omega_j[j]])*(Eomega2-encount_omega_2[list_omega_j[j]]) <=0:
                Mi[list_omega_j[j]][0] +=fore_real+aft_real
                Mi[list_omega_j[j]][1] +=fore_imag+aft_imag
    return Mi

def final_keisan(S,encount_omega_2,f):
    F = list(np.zeros(len(encount_omega_2)))
    #F_re=list(np.linspace(0,0,len(encount_omega_2)))
    #F_im=list(np.linspace(0,0,len(encount_omega_2)))
    #f=open("sample.txt","w",encoding="UTF-8")
    for j in range(n_angle*(n_lambda-1)):
        for i in range(len(encount_omega_2)):#出会い波周波数分
            F[i] += math.sqrt(S[j][i][0]**2+S[j][i][1]**2)
    for i in range(len(F)):
        f.write(str(F[i])+",")

    """0,1次モーメントをつくる"""
    m_0 , m_1 = 0.0 , 0.0
    return m_0,m_1   

def X_final_keisan1(S,encount_omega_2,f):
    """足し合わせたベクトルの角度"""
    #F = list(np.linspace(0,0,len(encount_omega_2)))
    F_Re=list(np.zeros(len(encount_omega_2)))
    F_Im=list(np.zeros(len(encount_omega_2)))
    #Re=0
    #Im=0
    for j in range(n_angle*(n_lambda-1)):
        for i in range(len(encount_omega_2)):#出会い波周波数分
            #F[i] += math.sqrt(S[j][i][0]**2+S[j][i][1]**2)
                F_Re[i]+=S[j][i][0]  
                F_Im[i]+=S[j][i][1]  

    for i in range(len(F_Re)):
        f.write(str(F_Re[i])+",")
    for i in range(len(F_Re)):
        f.write(str(F_Im[i])+",")
    #print(math.atan2(Im,Re))
    #Re_max=max(F_Re) 
    #Re_min=min(F_Re)
    #Im_max=max(F_Im)
    #Im_min=min(F_Im)
    #plt.plot([0,Re], [0,Im] , label='|F(k)|', marker='o', linestyle='-')
    #plt.xlim([min(Re,Im,0)-5,max(Re,Im,0)+5])
    #plt.ylim([min(Re,Im,0)-5,max(Re,Im,0)+5])
    #plt.plot(encount_omega_2, F , label='|F(k)|', marker='o', linestyle='-')  
    #plt.xlim([min(Re_min,Im_min)-5,max(Re_max,Im_max)+5])
    #plt.ylim([min(Re_min,Im_min)-5,max(Re_max,Im_max)+5])
    #plt.xlabel('ωe')
    #plt.ylabel('Se')#'#θ_xy[rad]
    #plt.grid()
    #plt.show()
    
    """0,1次モーメントをつくる"""
    m_0 , m_1 = 0.0 , 0.0
    return m_0,m_1   

def X_final_keisan2(S,encount_omega_2):
    """絶対値に角度を乗じた一次モーメントを0次モーメントで割る"""
    F = list(np.linspace(0,0,len(encount_omega_2)))
    F_Re=list(np.linspace(0,0,len(encount_omega_2)))
    F_Im=list(np.linspace(0,0,len(encount_omega_2)))
    Re=0
    Im=0
    thetaS=0.0
    Se=0.0
    for i in range(len(encount_omega_2)):#出会い波周波数分
        for j in range(n_angle*(n_lambda-1)):
            F[i] += math.sqrt(S[j][i][0]**2+S[j][i][1]**2)
            thetaS+=math.atan2(-S[j][i][1],S[j][i][0])*math.sqrt(S[j][i][0]**2+S[j][i][1]**2)
            Se     +=math.sqrt(S[j][i][0]**2+S[j][i][1]**2)
            F_Re[i]+=S[j][i][0]
            F_Im[i]+=-S[j][i][1]
    print(thetaS/Se)
    Re_max=max(F_Re) 
    Re_min=min(F_Re)
    Im_max=max(F_Im)
    Im_min=min(F_Im)

    plt.plot(F_Re, F_Im , label='|F(k)|', marker='o', linestyle='-')  
    plt.xlim([min(Re_min,Im_min)-5,max(Re_max,Im_max)+5])
    plt.ylim([min(Re_min,Im_min)-5,max(Re_max,Im_max)+5])
    plt.xlabel('Kxye')
    plt.ylabel('Qxye')#'#θ_xy[rad]
    plt.grid()
    plt.show()
    
    """0,1次モーメントをつくる"""
    m_0 , m_1 = 0.0 , 0.0
    for i in range(len(encount_omega_2)-1):
        m_0 += (F[i]+F[i+1])*(encount_omega_2[i+1]-encount_omega_2[i])/2  #台形則
        m_1 += (encount_omega_2[i+1]+encount_omega_2[i])/2*np.abs(F[i]+F[i+1])*(encount_omega_2[i+1]-encount_omega_2[i])/2
    return m_0,m_1   
        
def get_parameter_from_ind():
    row=random.randint(0, 118)
    Hv_1     =wave_statistics[row][2]+(wave_statistics[row][3]-wave_statistics[row][2])*random.random()#random.randint(20,895)/100 
    lambda_1 =random.random()*3.0
    theta_1  =random.random()*360
    s_1      =0#random.randint(1,30)
    omega_1  =2*np.pi*(1/wave_statistics[row][1]+(1/wave_statistics[row][0]-1/wave_statistics[row][1])*random.random())
    lambda_2 =0
    theta_2  =0
    s_2      =0
    omega_2  =0#
    Hv_2     =0#
    return lambda_1,theta_1,s_1,omega_1, Hv_1,lambda_2,theta_2,s_2,omega_2,Hv_2


def def_S_max(H,omega_m):
    HL=H/(1.56*4*math.pi**2/omega_m**2)
    """波集中度s1の周波数依存"""
    if HL<0.026:
        #print((HL))
        log_10=0.4444*math.log10(HL)+0.5849
        a=10**(log_10)
        S_max=12.99*a**(-2.5)
        
    elif HL>=0.026:
        log10_gF=(0.1507-math.sqrt(0.1507**2+4*0.005882*0.8789+4*0.005882*math.log10(HL)))/(2*0.005882)
        gF=10**log10_gF
        gT=1.37*(1-1/(1+0.008*gF**(1/3))**5)
        S_max=12.99*gT**2.5
    
    if S_max >=84:
        S_max=84
    return S_max

def spectrum_by_angle(lambda_1,theta_1,s_11,omega_1, Hv_1,lambda_2,theta_2,s_12,omega_2,Hv_2,omega,encount_angle,S_max1,S_max2):
 
    if omega<=omega_1:
        s_1=(S_max1*omega**5/omega_1**5)
    else:
        s_1=(S_max1*omega**(-2.5)/omega_1**(-2.5))
        
    spectrum_11=0.250*((4.0*lambda_1+1.0)/4.0*omega_1**4.0)**lambda_1
    spectrum_12=math.gamma(lambda_1)
    spectrum_16=Hv_1**2.0
    spectrum_13=omega**(4.0*lambda_1+1.0)
    spectrum_14=2.0**(2.0*s_1-1.0)*(math.gamma(s_1+1.0))**2.0/math.pi/math.gamma(2.0*s_1+1.0)
    spectrum_15=abs((math.cos((encount_angle-theta_1)/2.0*math.pi/180.0)))**(2.0*s_1)*math.exp(-(4.0*lambda_1+1.0)/4.0*(omega_1/omega)**4.0)
    spectrum_1=spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12

    
    #if omega<=omega_2:
        #s_2=(S_max2*omega**5/omega_2**5)
    #else:
        #s_2=(S_max2*omega**(-2.5)/omega_2**(-2.5))
        
   #spectrum_21=0.250*((4.0*lambda_2+1.0)/4.0*omega_2**4.0)**lambda_2
    #spectrum_22=math.gamma(lambda_2)
    #spectrum_26=Hv_2**2.0
    #spectrum_23=omega**(4.0*lambda_2+1.0)
    #spectrum_24=2.0**(2.0*s_2-1.0)*(math.gamma(s_2+1.0))**2.0/math.pi/math.gamma(2.0*s_2+1.0)
    #spectrum_25=abs((math.cos((encount_angle-theta_2)/2.0*math.pi/180.0)))**(2.0*s_2)*math.exp(-(4.0*lambda_2+1.0)/4.0*(omega_2/omega)**4.0)
    spectrum_2=0#spectrum_21/spectrum_23*spectrum_24*spectrum_25*spectrum_26/spectrum_22  
    
    return (spectrum_1+spectrum_2)*int(360/(n_angle))*math.pi/180.0      #5°ずつだから

    
def KeisanResponse(spectrum,line,Column_1,Column_2):
    #print(strip_data[line][Column_1])
    #print(strip_data[line][Column_1].dtype)
    X_Conjugate = strip_data[line][Column_1]*(math.cos(-strip_data[line][Column_1+1]*math.pi/180)+1j*math.sin(-strip_data[line][Column_1+1]*math.pi/180))
    Y           = strip_data[line][Column_2]*(math.cos(strip_data[line][Column_2+1]*math.pi/180)+1j*math.sin(strip_data[line][Column_2+1]*math.pi/180)) 
    return X_Conjugate*Y*spectrum 


if __name__ == "__main__":
    main()

start
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


In [11]:
import random
import numpy as np
#random.uniform(0,118)
random.randint(0, 3)

0